<a href="https://colab.research.google.com/github/Reynxzz/zyo-virtual-tour-guide/blob/main/Recommender_System_Zyo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_excel("hotel_bali.xlsx")
df.head(10)

,name,address,description
0,Umah Santi Residence,"Jalan Taman Paradise, 80361 Ungasan, Indonesia","Located in Ungasan, 2.9 km from Garuda Wisnu K..."
1,Umadhatu Resorts,Jalan Raya Denpasar Gilimanuk Desa Megati Sele...,"With garden views, Umadhatu Resorts is located..."
2,MaxOne Rejuvination @Gianyar,"Jl. Udayana , Gianyar - Bali, 80581 Balian, In...","MaxOne Rejuvination @Gianyar has a restaurant,..."
3,North Wing Canggu Resort,"No.79 Jalan Raya Tuka, 80361 Dalung, Indonesia","Located in Dalung, 9.2 km from Ubung Bus Stati..."
4,Tukad Gepuh Cottage Nusa Penida RedPartner,"Te tukad gepuh cottage banjar gepuh, desa, Tan...",Tukad Gepuh Cottage Nusa Penida RedPartner is ...
5,Lempuyang Boutique Hotel,"Jalan Raya Bukit Lempuyang Gang Kuum Sari, 808...","Located in Karangasem, 2.3 km from Ujung Beach..."
6,Rejeng Garden Camp,"Jln. Pendakian gunung Batur, Kintamani, Bali, ...","Located in Bangli, 27.4 km from Tegallalang Ri..."
7,Adhi Jaya Suite,"Jalan Jatiluwih Kawan, 82152 Jatiluwih, Indonesia","Adhi Jaya Suite is located in Jatiluwih, withi..."
8,Pertivi Love,"Jalan Raya Desa Sambangan, 81161 Ambengan, Ind...","Located in Ambengan in the Bali region, Pertiv..."
9,Swan Paradise by Desa Pramana Swan,"Banjar Banda, Desa Saba, Blahbatuh, 80581 Gian...",Swan Paradise by Desa Pramana Swan - CHSE Cert...


In [6]:
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         49 non-null     object
 1   address      49 non-null     object
 2   description  49 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [7]:
def print_description(index):
    example = df[df.index == index][['description', 'name', 'address']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama:', example[1])
        print('Alamat:', example[2])     

In [8]:
print_description(2)

MaxOne Rejuvination @Gianyar has a restaurant, outdoor swimming pool, a bar and garden in Balian. This 3-star hotel offers a concierge service and a tour desk. The property has a 24-hour front desk, airport transportation, room service and free WiFi. At the hotel, every room includes a closet. All rooms have air conditioning, a desk and a flat-screen TV, and some rooms at MaxOne Rejuvination @Gianyar have a balcony. Guests at the accommodation can enjoy a buffet breakfast. MaxOne Rejuvination @Gianyar has a sun terrace. Goa Gajah is 3.9 km from the hotel, while Tegenungan Waterfall is 4.3 km from the property. The nearest airport is Ngurah Rai International Airport, 35.4 km from MaxOne Rejuvination @Gianyar. Couples in particular like the location – they rated it 8.9 for a two-person trip.
Nama: MaxOne Rejuvination @Gianyar
Alamat: Jl. Udayana , Gianyar - Bali, 80581 Balian, Indonesia


In [ ]:
import nltk
nltk.download('popular')

In [11]:
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
stopworda = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text

# Buat kolom tambahan untuk data description yang telah dibersihkan   
df['desc_clean'] = df['description'].apply(clean_text)

In [12]:
df.head(5)

,name,address,description,desc_clean
0,Umah Santi Residence,"Jalan Taman Paradise, 80361 Ungasan, Indonesia","Located in Ungasan, 2.9 km from Garuda Wisnu K...",located ungasan 29 km garuda wisnu kencana uma...
1,Umadhatu Resorts,Jalan Raya Denpasar Gilimanuk Desa Megati Sele...,"With garden views, Umadhatu Resorts is located...",garden views umadhatu resorts located tabanan ...
2,MaxOne Rejuvination @Gianyar,"Jl. Udayana , Gianyar - Bali, 80581 Balian, In...","MaxOne Rejuvination @Gianyar has a restaurant,...",maxone rejuvination gianyar restaurant outdoor...
3,North Wing Canggu Resort,"No.79 Jalan Raya Tuka, 80361 Dalung, Indonesia","Located in Dalung, 9.2 km from Ubung Bus Stati...",located dalung 92 km ubung bus station north w...
4,Tukad Gepuh Cottage Nusa Penida RedPartner,"Te tukad gepuh cottage banjar gepuh, desa, Tan...",Tukad Gepuh Cottage Nusa Penida RedPartner is ...,tukad gepuh cottage nusa penida redpartner 1st...


In [13]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['desc_clean', 'name', 'address']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama:', example[1])
        print('Alamat:', example[2])

In [14]:
df.set_index('name', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['desc_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.04387469, 0.0507868 , ..., 0.06145423, 0.06197205,
        0.03053378],
       [0.04387469, 1.        , 0.04374968, ..., 0.1494559 , 0.03329416,
        0.02540588],
       [0.0507868 , 0.04374968, 1.        , ..., 0.05297025, 0.02819303,
        0.03830658],
       ...,
       [0.06145423, 0.1494559 , 0.05297025, ..., 1.        , 0.11379785,
        0.04114377],
       [0.06197205, 0.03329416, 0.02819303, ..., 0.11379785, 1.        ,
        0.01230094],
       [0.03053378, 0.02540588, 0.03830658, ..., 0.04114377, 0.01230094,
        1.        ]])

In [15]:
# Set index utama di kolom 'name'
indices = pd.Series(df.index)
indices[:50]

0                           Umah Santi Residence
1                               Umadhatu Resorts
2                   MaxOne Rejuvination @Gianyar
3                       North Wing Canggu Resort
4     Tukad Gepuh Cottage Nusa Penida RedPartner
5                       Lempuyang Boutique Hotel
6                             Rejeng Garden Camp
7                                Adhi Jaya Suite
8                                   Pertivi Love
9             Swan Paradise by Desa Pramana Swan
10                   Sri Abi Ratu Villa & Resort
11                Tirta Ayu Hotel and Restaurant
12                         Alam Terrace Cottages
13                Kiskenda Cottages & Restaurant
14          Keris Art Shop and Villa Telaga Wana
15                  Wana Shanti Villa by Madhava
16                          Alamdini Resort Ubud
17                    Puri Saraswati Dijiwa Ubud
18            Puri Dajuma Beach Eco-Resort & Spa
19                          Queen at Gunung Sari
20                To

In [16]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_hotel = []
    
    # Mengambil nama hotel berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    for i in top_10_indexes:
        recommended_hotel.append(list(df.index)[i])
        
    return recommended_hotel

In [17]:
recommendations('Umah Santi Residence')

['Ubud Aura Retreat',
 'Queen at Gunung Sari',
 'White dove guest house dua',
 'Lempuyang Boutique Hotel',
 'The Saren',
 'Jembrana Bali Homestay',
 'The Kalyana Ubud Resort',
 'RedDoorz Plus @ Singaraja',
 'North Wing Canggu Resort',
 'Alam Bali Lake View']